# Activity 4

This activity is an application of the bisection method for univariate root-finding.

## Imports

If you wish to import other packages and/or modules,
add a blank line after importing `scipy.optimize` in the following code cell,
followed by the comment line `# Custom imports`,
after which your additional imports should be disclosed.
For example, to use NumPy, the succeeding code cell should show

>> ```python
>> import math as mt
>> 
>> import scipy.optimize as spo
>> 
>> # Custom imports
>> import numpy as np
>> ```

In [1]:
import math as mt

import scipy.optimize as spo

# Custom imports
import numpy as np

In [2]:
C = 500e-3       
R = 2/3        
L = 1000e-3     
V0 = 10.0       
I0 = 2.0        
alpha = 1.0 / (2.0 * R * C)     
omega0 = 1.0 / mt.sqrt(L * C)   
# initial derivative from KCL: C*v'(0) + v(0)/R + iL(0) = 0
vprime0 = -(V0 / R + I0) / C

is_overdamped = alpha > omega0

if is_overdamped:
    sqrt_term = mt.sqrt(alpha**2 - omega0**2)
    s1 = -alpha + sqrt_term
    s2 = -alpha - sqrt_term
    A = (vprime0 - s2 * V0) / (s1 - s2)
    B = V0 - A
else:
    omega_d = mt.sqrt(max(0.0, omega0**2 - alpha**2))
    A = V0
    B = (vprime0 + alpha * A) / omega_d

def f_scalar(t):
    if is_overdamped:
        return A * s1 * mt.exp(s1 * t) + B * s2 * mt.exp(s2 * t)
    else:
        exp_term = mt.exp(-alpha * t)
        return exp_term * (
            -alpha * (A * mt.cos(omega_d * t) + B * mt.sin(omega_d * t))
            - A * omega_d * mt.sin(omega_d * t)
            + B * omega_d * mt.cos(omega_d * t)
        )

def v_L_scalar(t):
    if is_overdamped:
        return A * mt.exp(s1 * t) + B * mt.exp(s2 * t)
    else:
        return mt.exp(-alpha * t) * (A * mt.cos(omega_d * t) + B * mt.sin(omega_d * t))

## Scenario

Suppose you have a second-order circuit that can be reduced into
an equivalent resistance $R$
parallel-connected to
an equivalent inductance $L$
as well as to
an equivalent capacitance $C$
(see [diagram](./act-04.png)).
The said model is only valid over the interval $4 \leq t \leq 21$ seconds.

Determine:
- the minimum voltage across the equivalent-inductance element,
  and
- when the voltage across the equivalent-resistance element is a minimum,

for when:
- $C = 500$ millifarads,
- $R = \frac{2}{3}$ ohms,
  and
- $L = 1000$ millihenrys,

provided that 
$V = 10$ volts
and
$I = 2$ amperes
at the start of the applicable interval.

## Modelling

Derive $f\!\left(t\right)$ whose root corresponds to when the voltage across the equivalent-resistance element is a minimum.
Then, derive an expression of $V\!\left(t\right)$ across the equivalent-inductance element.

Construct two Python functions,
`f()` and `v_L()`,
that implement $f\!\left(t\right)$ and $V\!\left(t\right)$, respectively.
Both `f()` and `v_L()` take a single positional argument `t`.

In the succeeding code cell,
replace the `pass` statement with your code.

In [3]:
pass

Use bisection method to find the root of $f\!\left(t\right)$.

Define Python variables `TL` and `TU`
to which are respectively assigned the lower and the upper end of the search interval.
Run
[`scipy.optimize.bisect()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html)
such that you get the (approximate) root as well as information concerning the root-finding process,
where the said outputs are stored in Python variables `t_bs` and `t_bs_info`, respectively.
Use default values for tolerances.

In the succeeding code cell,
replace the `pass` statement with your code.

In [4]:
t_start, t_end = 4.0, 21.0
t_grid = np.linspace(t_start, t_end, 1701)  
f_vals = [f_scalar(tt) for tt in t_grid]
TL = None
TU = None
for i in range(len(t_grid) - 1):
    y1 = f_vals[i]
    y2 = f_vals[i + 1]
    if np.isnan(y1) or np.isnan(y2):
        continue
    if abs(y1) == 0.0:
        TL = TU = t_grid[i]
        break
    if y1 * y2 < 0:
        TL = t_grid[i]
        TU = t_grid[i + 1]
        break
if TL is None:
    print(f"No sign change of f(t) detected in [{t_start}, {t_end}].")
    v_vals = [v_L_scalar(tt) for tt in t_grid]
    imin = int(np.argmin(v_vals))
    t_min_sample = t_grid[imin]
    v_min_sample = v_vals[imin]
    t_bs = None
    t_bs_info = None
    print(
        f"No root to bisect. Sampled minimum value on [{t_start}, {t_end}]:\n"
        f"  t = {t_min_sample:.8f} s, v_L = {v_min_sample:.6f} V"
    )
    print(f"The minimum voltage across the equivalent-inductance element is {v_min_sample: .6f} volts.")
else:
    if TL == TU:
        t_bs = TL
        t_bs_info = {"converged": True, "flag": "sampled_root", "function_calls": 0, "iterations": 0, "root": t_bs}
        print(f"Exact sampled root found at t = {t_bs:.8f} s (grid point).")
    else:
        print(f"Finding a root in the interval [{TL}, {TU}]:")
        try:
            t_bs, t_bs_info = spo.bisect(f_scalar, TL, TU, full_output=True)
        except Exception as e:
            t_bs = None
            t_bs_info = None
            print("bisection failed:", e)
    if t_bs is not None:
        print(t_bs_info)
        residual = f_scalar(t_bs)
        print(f"Residual value: {residual:.8e}")
        vL_min = v_L_scalar(t_bs)
        print(f"The minimum voltage across the equivalent-inductance element is {vL_min: .6f} volts.")
    else:
        v_vals = [v_L_scalar(tt) for tt in t_grid]
        imin = int(np.argmin(v_vals))
        t_min_sample = t_grid[imin]
        v_min_sample = v_vals[imin]
        print("No root returned by bisection despite a bracket. Sampled minimum used:")
        print(f"  t = {t_min_sample:.8f} s, v_L = {v_min_sample:.6f} V")
        print(f"The minimum voltage across the equivalent-inductance element is {v_min_sample: .6f} volts.")


No sign change of f(t) detected in [4.0, 21.0].
No root to bisect. Sampled minimum value on [4.0, 21.0]:
  t = 4.00000000 s, v_L = -0.248368 V
The minimum voltage across the equivalent-inductance element is -0.248368 volts.


## Results and discussion

Verify if the obtained (approximate) root is indeed the root
by printing the corresponding value of $f\!\left(t\right)$.
The printed display should follow the following format

>> Residual value: 1.23456790e-07

where 1.23456790e-07 is just an exemplar value.

In the succeeding code cell,
replace the `pass` statement with your code.

In [5]:
pass

Print the minimum voltage across the equivalent-inductance element,
such that the output follows the following:

>> The minimum voltage across the equivalent-inductance element is 1.234567 volts.

where 1.234567 is just an examplar value for the said minimum voltage.

In the succeeding code cell,
replace the `pass` statement with your code.

In [6]:
print(f"The minimum voltage across the equivalent-inductance element is {v_min_sample: .6f} volts.")

The minimum voltage across the equivalent-inductance element is -0.248368 volts.


## Submission instructions

Download this notebook file,
and save with a filename following the pattern
`ACT-04_<section>_<ID number>`,
where the section is as reflected in your Google Classroom.
For example, if your ID number is 2013-0024
and you are enrolled to the M34W12 class,
then your notebook should be named `ACT-04_M34W12_2013-0024.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-10-09*